# Imports

In [29]:

import pandas as pd
import numpy as np
from datetime import datetime 
import requests
import math
import re

from bs4 import BeautifulSoup

# Collect data

* coleta os 36 prodcts da vitrini

In [30]:
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# url do site
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

# Beautifullsoup
soup = BeautifulSoup( page.text, 'html.parser' )

# ===================  Product Data  =================================
products = soup.find('ul', class_= 'products-listing small')
product_list = products.find_all('article', class_ = 'hm-product-item')

# Product_id
product_list[0].get('data-articlecode')
product_id = [i.get('data-articlecode') for i in product_list]

# Product_category
product_list[0].get('data-category')
product_category = [i.get('data-category') for i in product_list]

# product_name
product_list = products.find_all('a', class_= 'item-link' )
product_list[0].get('title')
product_name = [i.get('title') for i in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [i.get_text() for i in product_list]


data = pd.DataFrame([ product_id, product_category, product_name, product_price ]).T
data.columns = ['product_id','product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S:')
# #plotar
# data.head()

In [31]:
data.shape

(36, 5)

In [32]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1202825001,men_jeans_loose,Loose Jeans,$ 39.99,2023-10-19 21:37:36:
1,1130309007,men_jeans_loose,Baggy Jeans,$ 39.99,2023-10-19 21:37:36:
2,1096385001,men_jeans_loose,Loose Jeans,$ 29.99,2023-10-19 21:37:36:
3,1159764002,men_jeans_bootcut,Bootcut Loose Jeans,$ 49.99,2023-10-19 21:37:36:
4,0811993036,men_jeans_regular,Straight Regular Jeans,$ 39.99,2023-10-19 21:37:36:


# Data Collection by Product

* coleta dos prod sxpecifico entra em cada um e faz as rspagem 

In [45]:

# Dicionário que diz para o endpoint que o usuário é um Chrome (Browser)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

df_compositions = pd.DataFrame()

# Cria uma lista vazia para adicionar as colunas que não foram coletadas anteriormente, caso um novo produto tenha alguma característica não capturada antes
aux = []

# Suponha que você tenha um DataFrame chamado "data" com informações dos produtos, como 'product_id'
for i in range(len(data)):
    URL = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html' 
    print('Product: {}'.format(URL))  # Correção na formatação da URL

    # Requisita essa URL do servidor do site H&M
    page = requests.get(URL, headers=headers)

    # Parâmetro de entrada para o BeautifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')

    # ========================== Color Name ===============================
    
    # Product_color
    product_list = soup.find_all('a', class_='filter-option miniature')
    product_color = [i.get('data-color') for i in product_list]

    # Product_id
    product_id = [i.get('data-articlecode') for i in product_list]
    
    # Criando o DataFrame
    df_color = pd.DataFrame([ product_color,  product_id]).T
    df_color.columns = ['product_color', 'product_id']

    for j in range(len(df_color)):
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html' 
        print('Color:{}'.format(url))
        
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # ===============================  =======================================
        product_elements = soup.find_all('div', class_='content pdp-text pdp-content')
        
        # Inicialize uma lista para armazenar as informações de todos os produtos
        all_products_info = []
        
        for product_element in product_elements:

            product_info = {}

            # =============================== Product Price  =====================================
            price_tag = soup.find('span', class_='price-value')
            if price_tag:
                price = price_tag.get_text(strip=True)
                product_info['price'] = price
            
            # =============================== Composition =======================================
            # Extrair informações específicas para cada produto
            fit_element = product_element.find('dt', string='Fit')
            if fit_element:
                fit = fit_element.find_next('dd').text.strip()
                product_info['fit'] = fit
            
            size_element = product_element.find('dt', string='Size')
            if size_element:
                size = size_element.find_next('dd').text.strip()
                product_info['size'] = size
            
            composition_element = product_element.find('h3', string='Composition')
            if composition_element:
                composition = composition_element.find_next('ul').text.strip()
                product_info['composition'] = composition.replace('\n', '')

            ## =============================== product_name =======================================    
            product_name_element = product_element.find('h1', class_='primary product-item-headline')
            if product_name_element:
                product_name = product_name_element.text.strip()
                product_info['product_name'] = product_name
            
        
                            

            article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
            if article_number_element:
                article_number_text = article_number_element.text.strip()
                product_id = re.search(r'\d+', article_number_text).group()
                product_info['product_id'] = product_id
                product_info['product_color'] = df_color.loc[j, 'product_color']
                
            
            # Adicionar as informações do produto à lista
            all_products_info.append(product_info)

        # Criar o DataFrame
        df_composition = pd.DataFrame(all_products_info, columns=['product_name', 'price', 'fit', 'size', 'composition', 'product_id', 'product_color'])

        # Garantir o mesmo número de colunas
        df_composition = df_composition[['product_name', 'price', 'fit', 'size', 'composition', 'product_id', 'product_color']]
        
        # Atualizar a lista auxiliar com novas colunas
        aux = aux + df_composition.columns.tolist()

        # ========================== df_color + df_composition ==================================
        df_compositions = pd.concat([df_compositions, df_composition], axis=0)

df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

# Definir a coluna 'scrapy_datetime' com a data e hora atual
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


Product: https://www2.hm.com/en_us/productpage.1202825001.html
Product: https://www2.hm.com/en_us/productpage.1130309007.html
Color:https://www2.hm.com/en_us/productpage.1130309001.html
Color:https://www2.hm.com/en_us/productpage.1130309002.html
Color:https://www2.hm.com/en_us/productpage.1130309004.html
Color:https://www2.hm.com/en_us/productpage.1130309005.html
Color:https://www2.hm.com/en_us/productpage.1130309008.html
Color:https://www2.hm.com/en_us/productpage.1130309009.html
Color:https://www2.hm.com/en_us/productpage.1130309011.html
Product: https://www2.hm.com/en_us/productpage.1096385001.html
Color:https://www2.hm.com/en_us/productpage.1096385002.html
Product: https://www2.hm.com/en_us/productpage.1159764002.html
Color:https://www2.hm.com/en_us/productpage.1159764003.html
Color:https://www2.hm.com/en_us/productpage.1159764004.html
Color:https://www2.hm.com/en_us/productpage.1159764005.html
Product: https://www2.hm.com/en_us/productpage.0811993036.html
Color:https://www2.hm.com

In [68]:
soup.find('div', id_='inner')

In [34]:
df_compositions.shape

(359, 10)

In [46]:
df_compositions 

,product_name,price,fit,size,composition,product_id,product_color,style_id,color_id,scrapy_datetime
0,NaN,$27.99,Loose fit,NaN,"ShellCotton 100%Pocket liningPolyester 65%, Co...",1130309001,Denim black,1130309,001,2023-10-19 21:51:29
0,NaN,$29.99,Loose fit,NaN,"ShellCotton 100%Pocket liningPolyester 65%, Co...",1130309002,Light denim blue,1130309,002,2023-10-19 21:51:29
0,NaN,$39.99,Loose fit,"The model is 187cm/6'2"" and wears a size 31/32","ShellCotton 100%Pocket liningPolyester 65%, Co...",1130309004,Dark denim gray,1130309,004,2023-10-19 21:51:29
0,NaN,$39.99,Loose fit,"The model is 187cm/6'2"" and wears a size 31/32","ShellCotton 100%Pocket liningPolyester 65%, Co...",1130309005,Light denim blue,1130309,005,2023-10-19 21:51:29
0,NaN,$39.99,Loose fit,NaN,"ShellCotton 100%Pocket liningPolyester 65%, Co...",1130309008,Denim red,1130309,008,2023-10-19 21:51:29
...,...,...,...,...,...,...,...,...,...,...
0,NaN,$39.99,Relaxed fit,"The model is 186cm/6'1"" and wears a size 31/32",Cotton 100%,0875105046,Dark denim blue,0875105,046,2023-10-19 21:51:29
0,NaN,$13.99,Regular fit,"The model is 183cm/6'0"" and wears a size 31","ShellCotton 98%, Spandex 2%Pocket liningPolyes...",1166422003,White,1166422,003,2023-10-19 21:51:29
0,NaN,$13.99,Regular fit,"The model is 183cm/6'0"" and wears a size 31","ShellCotton 99%, Spandex 1%Pocket liningCotton...",1166422004,Denim gray,1166422,004,2023-10-19 21:51:29
0,NaN,$14.99,Regular fit,"The model is 183cm/6'0"" and wears a size 31","ShellCotton 99%, Spandex 1%Pocket liningPolyes...",1166422005,Denim blue,1166422,005,2023-10-19 21:51:29


In [44]:
df_composition

,product_name,price,fit,size,composition,product_id,product_color
0,NaN,$34.99,Regular fit,"The model is 189cm/6'2"" and wears a size 32","ShellCotton 98%, Spandex 2%Pocket liningPolyes...",1166422006,Black


In [42]:
product_info

{'price': '$34.99',
 'fit': 'Regular fit',
 'size': 'The model is 189cm/6\'2" and wears a size 32',
 'composition': 'ShellCotton 98%, Spandex 2%Pocket liningPolyester 65%, Cotton 35%',
 'product_id': '1166422006',
 'product_color': 'Black'}

# Data Cleaning

* limpeza e separacao dados:

In [35]:
## ================================== transfommacao das col ======================================

df_data = df_compositions.dropna(subset=['product_id'])

# product_name
#df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price - Remover o símbolo '$' e converter para float
df_data['price'] = df_data['price'].apply(lambda x: x.replace('$', '') if isinstance(x, str) else x ).astype(float)

# scrapy_datetime
# df_data['scrapy_datetime'] = pd.to_datetime(df_data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S:')

# Color Name
df_data['product_color'] = df_data['product_color'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if isinstance(x, str) else x)
#
# Fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# Size
df_data['size'] = df_data['size'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)
#
# size_number - Extraido números da coluna 'size'
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if isinstance(x, str) and re.search('\d{3}cm', x) else x)
df_data['size_number'] = df_data['size_number'].apply( lambda x: re.search('\d+', x).group(0) if pd.notnull( x ) else x )
#
# size_model - Extraido números da coluna 'size'
df_data['size_model'] = df_data['size'].apply( lambda x: re.search('\d+/\\d+', x).group(0) if isinstance(x, str) and re.search('\d+/\\d+', x) else x  )
#
# ========================== brack composition by comma =================================
#df1 = df_data['composition'].str.split(', ', expand=True).reset_index(drop=True) # onde tiver , quebro e tranforma em uma nova linha

# Dividir por vírgulas, aspas e o símbolo de %
df1 = df_data['composition'].str.split(r', |\'|%', expand=True).reset_index(drop=True)


# Substitua 'Shell' e 'Pocket lining' por uma string vazia na coluna 'composition'
df1[0] = df1[0].str.replace('Shell', '').str.replace('Pocket lining', '')
df1[1] = df1[1].str.replace('Shell', '').str.replace('Pocket lining', '').str.replace('Pocket', '').str.replace('Lining', '')
df1[2] = df1[2].str.replace('Shell', '').str.replace('Pocket lining', '')
df1[3] = df1[3].str.replace('Pocket lining', '').str.replace('Pocket', '').str.replace('Lining', '')
df1[5] = df1[5].str.replace('Pocket lining', '')

#criando um dataframe vazio do tamanho de df_data para alocar as colunas em ordem
df_ref = pd.DataFrame( index=np.arange( len( df_data ) ), columns=['cotton', 'Polyester','Spandex', 'Elastomultiester'] )

# Use expressões regulares para extrair columns=['Cotton', 'Polyester','Spandex', 'Elastomultiester'] em colunas separadas


# ======================================== DF Cotton =====================================================

# Usando expressão regular para extrair "Cotton" da coluna 0
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na=True), 0] # coleto dentro das minhas col[0] quem sao meus cotton = true/false = pass um loc e sel so a col 0
df_cotton_0.name = 'cotton'
# Usando expressão regular para extrair "Cotton" da coluna 1
df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na=True), 1]
df_cotton_1.name = 'cotton'
# Usando expressão regular para extrair "Cotton" da coluna 3
df_cotton_3 = df1.loc[df1[3].str.contains('Cotton', na=True), 3]
df_cotton_3.name = 'cotton'
# Usando expressão regular para extrair "Cotton" da coluna 5
df_cotton_5 = df1.loc[df1[5].str.contains('Cotton', na=True), 5]
df_cotton_5.name = 'cotton'
# Usando expressão regular para extrair "Cotton" da coluna 7
df_cotton_7 = df1.loc[df1[7].str.contains('Cotton', na=True), 7]
df_cotton_7.name = 'cotton'

# Use o método concat para combiná-los ao longo do eixo das colunas (axis=1)
df_cotton = pd.concat( [df_cotton_0, df_cotton_1, df_cotton_3, df_cotton_5, df_cotton_7], axis=0 )
df_ref.reset_index(drop=True, inplace=True)
df_cotton.reset_index(drop=True, inplace=True)
df_ref = pd.concat([df_ref, df_cotton], axis=1)

# ======================================== DF Polister =====================================================
df_poly_1 = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_poly_1.name = 'polyester'
df_poly_3 = df1.loc[df1[3].str.contains('Polyester', na=True), 3]
df_poly_3.name = 'polyester'

# Combine
df_ref.reset_index(drop=True, inplace=True)
df_poly = df_poly_1.combine_first( df_poly_3 )
df_ref = pd.concat([df_ref, df_poly], axis=1)
df_poly.reset_index(drop=True, inplace=True)


# ======================================== DF Spandex =====================================================
df_spandex_2 = df1.loc[df1[2].str.contains('Spandex', na=True), 2]
df_spandex_2.name = 'spandex'

df_spandex_4 = df1.loc[df1[4].str.contains('Spandex', na=True), 4]
df_spandex_4.name = 'spandex'

df_spandex_6 = df1.loc[df1[6].str.contains('Spandex', na=True), 6]
df_spandex_6.name = 'spandex'


df_spandex = pd.concat([ df_spandex_2, df_spandex_4, df_spandex_6 ], axis=0)

df_ref.reset_index(drop=True, inplace=True)
df_spandex.reset_index(drop=True, inplace=True)
df_ref = pd.concat([df_ref, df_spandex], axis=1)
#
#
# ======================================== DF Elastomultiester ==============================================

df_elastomultiester_2 = df1.loc[df1[2].str.contains( 'Elastomultiester', na=True), 2]
df_elastomultiester_2.name = 'elastomultiester' 

df_elastomultiester_4 = df1.loc[df1[4].str.contains( 'Elastomultiester', na=True ),4]
df_elastomultiester_4.name = 'elastomultiester'

# Combine
df_elastomultiester = df_elastomultiester_2.combine_first(df_elastomultiester_4)

df_ref = pd.concat( [ df_ref, df_elastomultiester], axis=1 )

# ======================================== DF 'Rayon =====================================================

df_rayon = df1.loc[ df1[2].str.contains('Rayon', na=True), 2 ]
df_rayon.name = 'Rayon'

df_ref = pd.concat( [ df_ref, df_rayon], axis=1).reset_index()


df_ref = df_ref.iloc[ :, ~df_ref.columns.duplicated( keep='last' ) ].reset_index(drop=True)

# join of combine with product
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True), df_ref ], axis=1)


# cotton
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
# Polyester
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
#spandex.
df_aux['spandex'] = df_aux['spandex'].apply( lambda x: int( re.search('\d+', x).group(0) ) /100 if pd.notnull(x) else x )
# Elastomultiester
df_aux['Elastomultiester'] = df_aux['Elastomultiester'].apply( lambda x: int(re.search('\d+', x).group(0))/ 100 if pd.notnull( x ) else x)
# Rayon
df_aux['Rayon'] = df_aux['Rayon'].apply( lambda x: int( re.search('\d+', x).group(0) ) /100 if pd.notnull(x) else x )

# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna( 0 )
df_data = pd.merge( df_data,  df_aux, on='product_id', how='left' )

# drop columns
df_data = df_data.drop(columns=['size', 'composition', 'index',	'Polyester', 'Spandex',	'Elastomultiester'], axis=1)

  # drop duplicate
df_data = df_data.drop_duplicates()                                                      

# <font color='F37126'> Data Insertion </font>

In [36]:
import sqlite3
from sqlalchemy import create_engine
# cron job roda duas vzs por semana


In [37]:
# RENOMEA AS COLUNAS E ORDENA 
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name', 
    'product_color',
    'fit', 
    'price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'elastomultiester',
    'Rayon',
    'scrapy_datetime'
]]

In [38]:
query_showroom_schema = """
    CREATE TABLE vitrine(
    product_id        TEXT,
    style_id          TEXT,
    color_id          TEXT,
    product_name      REAL, 
    product_color     TEXT,
    fit               TEXT, 
    price             REAL,
    size_number       TEXT,
    size_model        TEXT,
    cotton            REAL,
    polyester         REAL,
    spandex           REAL,
    elastomultiester  TEXT,
    Rayon             REAL,        
    scrapy_datetime   TEXT
    
    )
"""
# creat Table
conn = sqlite3.connect('database_hm.sqlite')
# Criara um cursor que execulta a query e cria a table que guardara os dados 
cursor = conn.execute( query_showroom_schema)
# usar o cursor executar
conn.commit()



In [39]:
# creat  Database connect
conn = create_engine('sqlite:///database_hm.sqlite', echo=False)

# Data insurt
data_insert.to_sql('vitrine', con=conn, if_exists='append', index=False)



148

In [40]:
query = """ 
SELECT FROM vitrine
"""

In [41]:
# # Inicialize uma lista para armazenar os DataFrames processados
# cotton_dataframes = []

# # Colunas onde você deseja procurar 'Cotton'
# columns_to_search = [0, 1, 3, 5, 7]

# for col in columns_to_search:
#     # Use expressão regular para extrair "Cotton" da coluna
#     cotton_df = df1.loc[df1[col].str.contains('Cotton', na=True), col]
#     # Defina o nome da coluna como 'cotton'
#     cotton_df.name = 'cotton'
#     # Adicione o DataFrame processado à lista
#     cotton_dataframes.append(cotton_df)

# # Use o método concat para combiná-los ao longo do eixo das colunas (axis=1)
# df_cotton = pd.concat(cotton_dataframes, axis=0)

# # Reset o índice se necessário
# df_cotton.reset_index(drop=True, inplace=True)